In [197]:
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

In [198]:
#stockin=input('Enter Stock to be evaluated')
#n=int(input("WIndow duration"))
stockin='AAPL'
n=20

ticker=yf.Ticker(stockin)
today = datetime.now()
prev_day = today - timedelta(days=1)
prev_day_str = prev_day.strftime('%Y-%m-%d')

hist = ticker.history(start=prev_day_str, end=today.strftime('%Y-%m-%d'), interval='1d')

if not hist.empty:
    prev_open = hist['Open'].iloc[0]
    prev_close = hist['Close'].iloc[0]
    print(f"Previous day open: {prev_open}")
    print(f"Previous day close: {prev_close}")
else:
    print("No data for previous day.")

Previous day open: 199.72999572753906
Previous day close: 196.4499969482422


In [199]:
prev_day_mean=(prev_close+prev_open)/2
prev_day_mean

198.08999633789062

In [200]:
df = yf.download(tickers=stockin, period='1d', interval='1m')

df.dropna(inplace=True)

df['RollingMean_n'] = df['Close'].shift(1).rolling(window=n).mean()
df['RollingStd_n'] = df['Close'].shift(1).rolling(window=n).std()

current_price = df['Close'].iloc[-1]
rolling_mean_nmin = df['RollingMean_n'].iloc[-1]
rolling_std_nmin = df['RollingStd_n'].iloc[-1]

print("Current Price:          ", current_price)
print(f"{n}-min Rolling Mean:    ", rolling_mean_nmin)
print(f"{n}-min Rolling Std Dev: ", rolling_std_nmin)

C:\Users\sauri\AppData\Local\Temp\ipykernel_22896\312912579.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=stockin, period='1d', interval='1m')
[*********************100%***********************]  1 of 1 completed

Current Price:           Ticker
AAPL    196.449997
Name: 2025-06-13 19:59:00+00:00, dtype: float64
20-min Rolling Mean:     196.04245986938477
20-min Rolling Std Dev:  0.16659999409433493


In [201]:
z_score=(current_price-rolling_mean_nmin) / rolling_std_nmin
max_vol_nmin = df['Volume'].shift(1).tail(n).max()
current_volume = df['Volume'].iloc[-1]

print("Current Tick Volume:", current_volume)
print(f"Max Volume in Last {n} Min (excluding current tick):", max_vol_nmin)
print("z-score",z_score)

Current Tick Volume: Ticker
AAPL    1003296
Name: 2025-06-13 19:59:00+00:00, dtype: int64
Max Volume in Last 20 Min (excluding current tick): Ticker
AAPL    415271.0
dtype: float64
z-score Ticker
AAPL    2.446201
Name: 2025-06-13 19:59:00+00:00, dtype: float64


In [202]:
amount = 10000
initial_amount = amount
position = 0
shares_held = 0
buy_percentage = 0.90
sell_percentage = 0.30
target_profit = initial_amount * 0.20

In [203]:
def buy_sig(z_score, max_vol_nmin, current_volume):
    z_score = float(z_score.iloc[-1]) if hasattr(z_score, 'iloc') else float(z_score)
    max_vol_nmin = float(max_vol_nmin.iloc[-1]) if hasattr(max_vol_nmin, 'iloc') else float(max_vol_nmin)
    current_volume = float(current_volume.iloc[-1]) if hasattr(current_volume, 'iloc') else float(current_volume)
    
    signal = 0
    signal_type = "HOLD"
    
    volume_margin = abs(current_volume - max_vol_nmin) / max_vol_nmin <= 0.15
    
    if z_score <= -0.3:
        signal = 1
        signal_type = "BUY"
    elif z_score > 1.3:
        signal = -1  
        signal_type = "SELL"
    elif volume_margin and (z_score <= 1.3) and (z_score>-0.3):
            signal = -1
            signal_type = "SELL"
    
         
    
    print(f"\nSignal Analysis:")
    print(f"Z-Score: {z_score:.4f}")
    print(f"Signal: {signal} ({signal_type})")
    print(f"Current Volume: {current_volume}")
    print(f"Max Volume ({n}min): {max_vol_nmin}")
    print(f"Volume within 15% margin: {volume_margin}")
    
    return signal

### BACKTESTING


In [204]:
def backtest_strategy(symbol, start_date, end_date, initial_capital=10000):
    df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
    
    if df.empty:
        print('No data available for the specified period')
        return
    
    amount = initial_capital
    initial_amount = amount
    position = 0
    shares_held = 0
    trades = []
    
    df['RollingMean_n'] = df['Close'].shift(1).rolling(window=n).mean()
    df['RollingStd_n'] = df['Close'].shift(1).rolling(window=n).std()
    
    df = df.dropna()
    
    for i in range(len(df)):
        current_price = float(df['Close'].iloc[i])
        rolling_mean_nmin = df['RollingMean_n'].iloc[i]
        rolling_std_nmin = df['RollingStd_n'].iloc[i]
        
        z_score = (current_price - rolling_mean_nmin) / rolling_std_nmin
        
        if i >= n:
            recent_volumes = df['Volume'].iloc[i-n:i]
            max_vol_nmin = recent_volumes.max()
        else:
            max_vol_nmin = df['Volume'].iloc[:i].max()
        
        current_volume = df['Volume'].iloc[i]
        
        signal = buy_sig(z_score, max_vol_nmin, current_volume)
        
        if signal == 1 and position == 0:  
            investment = amount * buy_percentage
            shares_to_buy = int(investment // current_price)
            if shares_to_buy > 0:
                position = current_price * shares_to_buy
                shares_held = shares_to_buy
                amount -= position
                trades.append({
                    'time': df.index[i],
                    'type': 'BUY',
                    'shares': shares_to_buy,
                    'price': current_price,
                    'value': position,
                    'remaining_cash': amount
                })

        elif signal == -1 and position > 0:  
            shares_to_sell = max(1, int(np.ceil(shares_held * sell_percentage))) if shares_held > 0 else 0
            if shares_to_sell > 0:
                sale_value = current_price * shares_to_sell
                amount += sale_value
                shares_held -= shares_to_sell
                position = current_price * shares_held
                trades.append({
                    'time': df.index[i],
                    'type': 'SELL',
                    'shares': shares_to_sell,
                    'price': current_price,
                    'value': sale_value,
                    'remaining_cash': amount
                })
        
        if shares_held > 0:
            position = current_price * shares_held
    
    total_value = amount + position
    total_return = ((total_value - initial_capital) / initial_capital) * 100
    
    print(f'\nBacktest Results ({start_date} to {end_date}):')
    print(f'Initial Capital: ${initial_amount:,.2f}')
    print(f'Final Capital: ${total_value:,.2f}')
    print(f'Total Return: {total_return:.2f}%')
    print(f'Number of Trades: {len(trades)}')
    
    print('\nTrade History:')
    for trade in trades:
        print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ ${trade['price']:.2f}")
    
    return df, trades

In [205]:
from datetime import datetime, timedelta

end_date = datetime.now()
start_date = end_date - timedelta(days=4)  

df, trades = backtest_strategy(stockin, start_date, end_date, initial_capital=10000)

C:\Users\sauri\AppData\Local\Temp\ipykernel_22896\2108613513.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
[*********************100%***********************]  1 of 1 completed
C:\Users\sauri\AppData\Local\Temp\ipykernel_22896\2108613513.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])



Signal Analysis:
Z-Score: -0.6565
Signal: 1 (BUY)
Current Volume: 124321.0
Max Volume (20min): nan
Volume within 15% margin: False

Signal Analysis:
Z-Score: -0.5841
Signal: 1 (BUY)
Current Volume: 113555.0
Max Volume (20min): 124321.0
Volume within 15% margin: True

Signal Analysis:
Z-Score: -0.5352
Signal: 1 (BUY)
Current Volume: 89595.0
Max Volume (20min): 124321.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: 0.4293
Signal: -1 (SELL)
Current Volume: 111615.0
Max Volume (20min): 124321.0
Volume within 15% margin: True

Signal Analysis:
Z-Score: -2.1475
Signal: 1 (BUY)
Current Volume: 234930.0
Max Volume (20min): 124321.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -1.5822
Signal: 1 (BUY)
Current Volume: 103718.0
Max Volume (20min): 234930.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -0.3952
Signal: 1 (BUY)
Current Volume: 87846.0
Max Volume (20min): 234930.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -0.0711
Signal: 0 (HOL